In [1]:
import numpy as np
import pandas as pd
from tdc.multi_pred import MTI
import torch
import torch.nn as nn
import torch.optim as optim
import BinaryClassifier
import torchtext
from transformers import BertTokenizer

In [2]:
data = MTI(name = 'miRTarBase')
neg_data = data.neg_sample()
neg_data = neg_data.get_data()

Found local copy...
Loading...
Done!


Only using about 1/4 the data because I can't run the server on my laptop and it's very slow with  all the data.

In [3]:
neg_data = neg_data[:round(len(neg_data)/4)]

Using the Tokenizer takes a long time but the benefit is that it automatically helps us do padding and gives us an attention mask.

In [4]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case = False)

In [31]:
mirna = [' '.join(mirna) for mirna in neg_data.miRNA.values]
targets = [' '.join(targets) for targets in neg_data.Target.values]

In [6]:
neg_data.miRNA

0           UGUGUUCUCAGGUCACCCCUG
1           UGUGUUCUCAGGUCACCCCUG
2           UGUGUUCUCAGGUCACCCCUG
3           UGUGUUCUCAGGUCACCCCUG
4           UGUGUUCUCAGGUCACCCCUU
                   ...           
200036     AACACACCUAUUCAAGGAUUCA
200037     AACACACCUGGUUAACCUCUUU
200038      CACACACACACACACACGUAU
200039    UUUGGGAUUGACGCCACAUGUCU
200040      CUUGGGAGCCCUGUUAGACUC
Name: miRNA, Length: 200041, dtype: object

In [9]:
string = 'UGUGUUCUCAGGUCACCCCUG'
string = ' '.join(string)
tokenizer(string, padding = True)
#tokenized_targets = tokenizer(targets, padding = True)

{'input_ids': [2, 26, 7, 26, 7, 26, 26, 23, 26, 23, 6, 7, 7, 26, 23, 6, 23, 23, 23, 23, 26, 7, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [216]:
for dataset in ['test', 'valid', 'train']:

    neg_data_split[dataset]['miRNA_ord'] = neg_data_split[dataset].apply(lambda x: (transformer(x["miRNA"])), axis = 1)
    neg_data_split[dataset]['Target_ord'] = neg_data_split[dataset].apply(lambda x: (transformer(x["Target"])), axis = 1)

Defining training, validation, and test sets

In [217]:
x_train, y_train = neg_data_split['train'].loc[:, ['miRNA_ord','Target_ord']], neg_data_split['train'].loc[:, 'Y']
x_valid, y_valid = neg_data_split['valid'].loc[:, ['miRNA_ord','Target_ord']], neg_data_split['valid'].loc[:, 'Y']
x_test, y_test = neg_data_split['test'].loc[:, ['miRNA_ord','Target_ord']], neg_data_split['test'].loc[:, 'Y']

In [246]:
tokenizer

BertTokenizer(name_or_path='Rostlab/prot_bert', vocab_size=30, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [247]:
tokenizer(['A B C D E', 'A D E C'], padding = True)

{'input_ids': [[2, 6, 27, 23, 14, 9, 3], [2, 6, 14, 9, 23, 3, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0]]}

In [ ]:
# Load the data
train_data = torch.tensor(np.load('train_data.npy')).float()
train_labels = torch.tensor(np.load('train_labels.npy')).float()
test_data = torch.tensor(np.load('test_data.npy')).float()
test_labels = torch.tensor(np.load('test_labels.npy')).float()

# Initialize the model and optimizer
model = BinaryClassifier()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(train_data)
    loss = nn.BCELoss()(outputs, train_labels)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print training progress
    if epoch % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Evaluate the model on test data
with torch.no_grad():
    outputs = model(test_data)
    predicted = (outputs > 0.5).float()
    accuracy = (predicted == test_labels).float().mean()
    print('Test Accuracy: {:.2f}%'.format(accuracy * 100))
